# 공원데이터분석

In [1]:
# !pip install dbfread
# !pip install haversine
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from haversine import haversine
from tqdm.notebook import tqdm
%matplotlib inline
import platform
import pickle
import re
platform.system()
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'

In [57]:
bus = pd.read_excel(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울시버스정류소좌표데이터(2020.03.06).xlsx', encoding='cp949')
park = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울시 주요 공원현황.csv', encoding='cp949')
office = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\전국공공시설개방정보표준데이터.csv', encoding='cp949')
# lib = pd.read_excel(r'C:\Users\이규호\NLP\github\seoul_contest\data\전국도서관표준데이터-20200926.xls', encoding='cp949')
# sports = pd.read_csv()

In [3]:
park

,공원번호,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107000,4.503956e+05,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do
1,3,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120000,4.488527e+05,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do
2,4,서울대공원,서울대공원은 세계 각국의 야생동물들이 살아 숨 쉬는 서울동물원과 다양한 재미와 즐거...,9132690m²,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,NaN,NaN,지하철 4호선 대공원역 하차(2번 출구) 도보 15분 정도 문의처 02)500-73...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,과천시,경기도 과천시 대공원광장로 102,전략기획실,02-500-7032,200994.267000,4.371640e+05,127.019847,37.426449,http://grandpark.seoul.go.kr/
3,5,서울숲,당초 골프장 승마장 등이 있던 뚝섬일대를 주거업무 지역으로 개발할 경우 약 4조원에...,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,수 목 : 소나무 섬잣나무 계수나무 외 95종 415795주 식물원 : 선인장 등 ...,http://parks.seoul.go.kr/template/common/img/p...,NaN,지하철 분당선 서울숲역 3번출구 도보로 약5분 2호선 뚝섬역 8번출구 도보로 약 1...,http://parks.seoul.go.kr/file/info/view.do?fId...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),서울숲 관리사무소,02-460-2905,203695.432000,4.492907e+05,127.041798,37.543072,http://parks.seoul.go.kr/template/sub/seoulfor...
4,6,월드컵공원,월드컵공원은 서울의 서쪽에 위치하여 1978년부터 1993년까지 15년간 서울시민이...,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,NaN,http://parks.seoul.go.kr/template/common/img/p...,지하철 6호선 월드컵경기장역 하차 → 1번 출구로 나온 후 직진 → 큰길(도로)이 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),월드컵공원,02-300-5500,190658.070000,4.515988e+05,126.878907,37.571805,http://parks.seoul.go.kr/template/sub/worldcup...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,1378,용마도시자연공원(사가정공원),2005년 4월 13일 개장한 사가정공원은 면목동 산 50번지 일대의 면목약수터지구...,109635㎡,2005.4.13,피크닉장 놀이터 체력단련장 체육관,NaN,NaN,NaN,NaN,http://parks.seoul.go.kr/file/info/view.do?fId...,중랑구,서울특별시 중랑구 면목7동 산50,중랑구청 공원녹지과,02-2094-2993,NaN,NaN,127.095997,37.579762,NaN
128,1379,문화비축기지,매봉산자락에 위치한 문화비축기지는 폐산업시설인 '마포석유비축기지'를 재생하여 역사적...,140022㎡,2017.9.1,6개의탱크와 문화마당 녹지,NaN,http://parks.seoul.go.kr/upload/seditorMulti/2...,NaN,? 일반사항 - 공원은 연중무휴이나 전시관은 월요일 휴관입니다. - 공원을 ...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 증산로 87,문화비축기지,02-376-8410,114264.460400,3.712093e+06,126.893245,37.571718,http://parks.seoul.go.kr/template/sub/culturet...
129,1380,경춘선숲길,일제 강점기때 우리 민족의 자본으로 만든 최초의 철도시설로 1939년 7월 25일 ...,211392㎡,NaN,조경시설 : 텃밭 잔디광장 철길 산책로 등 건축시설 : 무궁화호 객차(방문자센터) ...,NaN,NaN,지하철 이용시 지하철 7호선 하계역 4번출구 미성/장미아파트 도로 450m직진 지하...,○ 6.3km (시 작 점 : 육사삼거리 중간지점 : 화랑대사거리 → 화랑대...,http://parks.seoul.go.kr/file/info/view.do?fId...,NaN,노원구 공릉2동 산 82-2,중부공원녹지사업소,02-2289-4012,127.076482,3.762708e+01,127.076482,37.627077,NaN
130,1381,율현공원,? 율현공원은 보금자리주택 세곡2지구 사업의 결과물로 만들어졌다. 공원은 주택지구의...,157535㎡,2016.11.1,관리시설 : 사무실 조경시설 : 정자(4개) 조형쉼터(10개) 조형파고라(3개) 수...,수목 : 소나무 이팝나무 등 110종 143069주 초화 : 맥문동 벌개미취 등...,http://parks.seoul.go.kr/upload/seditorMulti/2...,NaN,공원 인근에 서울공항 활주로가 위치하고 있어 공원 내에서 드론을 날리거나 헬륨가스 ...,http://parks.seoul.go.kr/file/info/view.do?fId...,강남구,서울특별시 강남구 율현동 밤고개로24길(율현동 56),동부공원녹지사업소,02-459-9

In [4]:
park.columns

Index(['공원번호', '공원명', '공원개요', '면적', '개원일', '주요시설', '주요식물', '안내도', '오시는길',
       '이용시참고사항', '이미지', '지역', '공원주소', '관리부서', '전화번호', 'X좌표(GRS80TM)',
       'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '바로가기'],
      dtype='object')

In [5]:
park_df = park[['공원번호','공원명','면적','개원일','주요시설','지역','공원주소','X좌표(WGS84)','Y좌표(WGS84)']]
park_df

,공원번호,공원명,면적,개원일,주요시설,지역,공원주소,X좌표(WGS84),Y좌표(WGS84)
0,1,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,중구,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,3,길동생태공원,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,4,서울대공원,9132690m²,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,과천시,경기도 과천시 대공원광장로 102,127.019847,37.426449
3,5,서울숲,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,6,월드컵공원,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805
...,...,...,...,...,...,...,...,...,...
127,1378,용마도시자연공원(사가정공원),109635㎡,2005.4.13,피크닉장 놀이터 체력단련장 체육관,중랑구,서울특별시 중랑구 면목7동 산50,127.095997,37.579762
128,1379,문화비축기지,140022㎡,2017.9.1,6개의탱크와 문화마당 녹지,마포구,서울특별시 마포구 증산로 87,126.893245,37.571718
129,1380,경춘선숲길,211392㎡,NaN,조경시설 : 텃밭 잔디광장 철길 산책로 등 건축시설 : 무궁화호 객차(방문자센터) ...,NaN,노원구 공릉2동 산 82-2,127.076482,37.627077
130,1381,율현공원,157535㎡,2016.11.1,관리시설 : 사무실 조경시설 : 정자(4개) 조형쉼터(10개) 조형파고라(3개) 수...,강남구,서울특별시 강남구 율현동 밤고개로24길(율현동 56),127.115594,37.472332


공원번호는 전국 기준인듯하고, 면적은 몇몇개는 re써서 뽑아야할듯. 개원일은 큰 의미있을지는 모르겠다.주요시설에선 valuecount랑 unique본다음에 특수점으로 넣을지 생각하고, 지역이 대부분있는듯하지만 없으면 주소에서 뽑아야할듯. 

In [6]:
park_need = park_df[['공원명','X좌표(WGS84)','Y좌표(WGS84)', '면적']]
park_need.면적 = park_need.면적.astype(str)
park_need

C:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,공원명,X좌표(WGS84),Y좌표(WGS84),면적
0,남산도시자연공원,126.990377,37.550140,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡
1,길동생태공원,127.154779,37.540394,80683㎡
2,서울대공원,127.019847,37.426449,9132690m²
3,서울숲,127.041798,37.543072,480994㎡
4,월드컵공원,126.878907,37.571805,2284085㎡
...,...,...,...,...
127,용마도시자연공원(사가정공원),127.095997,37.579762,109635㎡
128,문화비축기지,126.893245,37.571718,140022㎡
129,경춘선숲길,127.076482,37.627077,211392㎡
130,율현공원,127.115594,37.472332,157535㎡


In [7]:
park_need.head(40)

,공원명,X좌표(WGS84),Y좌표(WGS84),면적
0,남산도시자연공원,126.990377,37.550140,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡
1,길동생태공원,127.154779,37.540394,80683㎡
2,서울대공원,127.019847,37.426449,9132690m²
3,서울숲,127.041798,37.543072,480994㎡
4,월드컵공원,126.878907,37.571805,2284085㎡
5,광화문시민열린마당,126.978023,37.575045,8948.1㎡
6,진관근린공원(구파발폭포),126.919216,37.637024,휴양 및 편익시설 : 평의자 7개 화장실 23.18㎡ 조경시설 - 은하폭포(높이 1...
7,중마루근린공원,126.911267,37.519832,6456㎥
8,응봉공원,127.021765,37.557228,80309㎡ 배수지 시설용량 : 201.000㎥ 송수 펌프용량 : 직송급수계통 30...
9,매화근린공원,126.861150,37.557583,1980.4㎡


In [55]:
for i in range(len(park_need.면적)):
#     print(re.findall('([1-9]*?\.*[1-9]+?)㎡+?m*?²*?', park_need.면적[i]))
    print(re.findall('([0-9]+?\.?[0-9]?)㎡|([0-9]+?\.?[0-9]?)m²|([0-9]+?\.?[0-9]?)㎥', park_need.면적[i]))
    
     

[('2896887', '', ''), ('2454140', '', ''), ('442747', '', '')]
[('80683', '', '')]
[('', '9132690', '')]
[('480994', '', '')]
[('2284085', '', '')]
[('8948.1', '', '')]
[('18', '', ''), ('255', '', ''), ('90', '', ''), ('102', '', ''), ('55', '', ''), ('03', '', ''), ('36', '', '')]
[('', '', '6456')]
[('80309', '', ''), ('', '', '000'), ('', '', '30000'), ('', '', '16400')]
[('1980.4', '', '')]
[('5197.7', '', '')]
[('11467', '', '')]
[('229539', '', '')]
[]
[('', '', '60')]
[('61544', '', ''), ('40792', '', ''), ('6600', '', ''), ('14152', '', '')]
[('424106', '', '')]
[('560552', '', '')]
[('', '', '26696.8')]
[('75900', '', '')]
[]
[('', '', '201779')]
[('258991', '', '')]
[('08', '', '')]
[('', '', '171294.1')]
[('15000', '', '')]
[('297926', '', '')]
[('411972', '', '')]
[('98470', '', '')]
[('2038741.3', '', '')]
[('603715.2', '', '')]
[('108120', '', '')]
[('', '', '29619.3')]
[('29843.8', '', '')]
[('126026', '', '')]
[('135002.7', '', '')]
[('384026', '', '')]
[('', '', '1800

공원 넓이를 표현하는 방식이 다 다르다
그냥 1000제곱미터(㎡)라고 한곳이 있는 반면, 1000세제곱미터(㎥)로 한곳도, 1000띄어쓰기제곱미터(㎡) 한곳, 1000미터와 제곱(m²) 

배수지용량은 왜나오는데..;

# 버스

In [9]:
bus

,표준ID,ARS-ID,정류장명,X좌표,Y좌표,비고
0,100000001,1001,종로2가사거리,126.987750,37.569765,NaN
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,NaN
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,NaN
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,NaN
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,NaN
...,...,...,...,...,...,...
11275,124000334,25995,우성아파트,127.139339,37.550386,NaN
11276,124000333,25996,우성아파트,127.140046,37.550643,NaN
11277,124000332,25997,조일약국,127.123596,37.533630,NaN
11278,124000331,25998,성내시장,127.125497,37.536155,NaN


In [10]:
del bus['비고']

In [11]:
bus

,표준ID,ARS-ID,정류장명,X좌표,Y좌표
0,100000001,1001,종로2가사거리,126.987750,37.569765
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671
3,100000004,1004,종로2가.삼일교,126.987613,37.568579
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243
...,...,...,...,...,...
11275,124000334,25995,우성아파트,127.139339,37.550386
11276,124000333,25996,우성아파트,127.140046,37.550643
11277,124000332,25997,조일약국,127.123596,37.533630
11278,124000331,25998,성내시장,127.125497,37.536155


In [12]:
bus_df = bus[['표준ID','정류장명', 'X좌표', 'Y좌표']]
bus_df

,표준ID,정류장명,X좌표,Y좌표
0,100000001,종로2가사거리,126.987750,37.569765
1,100000002,창경궁.서울대학교병원,126.996566,37.579183
2,100000003,명륜3가.성대입구,126.998340,37.582671
3,100000004,종로2가.삼일교,126.987613,37.568579
4,100000005,혜화동로터리.여운형활동터,127.001744,37.586243
...,...,...,...,...
11275,124000334,우성아파트,127.139339,37.550386
11276,124000333,우성아파트,127.140046,37.550643
11277,124000332,조일약국,127.123596,37.533630
11278,124000331,성내시장,127.125497,37.536155


# 동사무소(행정복지센터)

In [59]:
office

,개방시설명,개방장소명,개방시설유형구분,휴관일,평일운영시작시각,평일운영종료시각,주말운영시작시각,주말운영종료시각,유료사용여부,사용기준시간,...,소재지지번주소,관리기관명,담당부서명,사용안내전화번호,홈페이지주소,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,아르코공연연습센터 광주세미나실,소촌아트팩토리,강의실,월,10:00,18:00,10:00,18:00,N,NaN,...,광주광역시 광산구 소촌동 845,광산구청,문화예술과,062-960-3688,https://www.arko.or.kr/infra/,35.152627,126.791234,2020-07-20,3630000,광주광역시 광산구
1,아르코공연연습센터 광주리딩룸,소촌아트팩토리,강의실,월,10:00,18:00,10:00,18:00,N,NaN,...,광주광역시 광산구 소촌동 845,광산구청,문화예술과,062-960-3688,https://www.arko.or.kr/infra/,35.152627,126.791234,2020-07-20,3630000,광주광역시 광산구
2,송광종합사회복지관 소강당,송광종합사회복지관,강당,주말,09:00,18:00,00:00,00:00,Y,1,...,광주광역시 광산구 우산동 1603-1,광산구청,공동체복지과,062-941-8248,www.gwangsan.go.kr,35.159824,126.804145,2020-07-20,3630000,광주광역시 광산구
3,송광종합사회복지관 한사랑방,송광종합사회복지관,강의실,주말,09:00,18:00,00:00,00:00,Y,1,...,광주광역시 광산구 우산동 1603-1,광산구청,공동체복지과,062-941-8248,www.gwangsan.go.kr,35.159824,126.804145,2020-07-20,3630000,광주광역시 광산구
4,송정1동 행정복지센터 다목적실(2층),송정1동 행정복지센터,다목적실,주말,09:00,17:00,00:00,00:00,N,NaN,...,광주광역시 광산구 송정동 756-1,광산구청,송정1동 행정복지센터,062-960-7942,www.gwangsan.go.kr,35.140746,126.798783,2020-07-20,3630000,광주광역시 광산구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6717,여수문화홀,여수문화홀,강당,"행사없는 토,일요일,공휴일",08:00,22:00,08:00,22:00,Y,4,...,NaN,전라남도 여수시청,문화예술과,061-659-4692,NaN,34.760520,127.662136,2019-06-17,4810000,전라남도 여수시
6718,해오름전시실,해오름전시실,전시실,월요일,09:00,18:00,09:00,18:00,N,0,...,NaN,전라남도 여수시청,문화예술과,061-666-3766,NaN,34.739025,127.748151,2019-06-17,4810000,전라남도 여수시
6719,상주문화회관,상주문화회관,기타,명절,09:00,22:00,09:00,22:00,Y,13,...,NaN,경상북도 상주시청,문화예술과,054-537-6215,http://www.sangju.go.kr/culture/index.jsp,36.411915,128.159163,2019-05-27,5110000,경상북도 상주시
6720,청소년수련관,청소년수련관,풋살경기장,"명절(신정,구정, 추석)",09:00,21:00,09:00,18:00,Y,1,...,NaN,경상북도 상주시청,평생학습원,054-537-5562,http://youth.sangju.go.kr,36.429813,128.162789,2019-05-27,5110000,경상북도 상주시


In [61]:
office.columns

Index(['개방시설명', '개방장소명', '개방시설유형구분', '휴관일', '평일운영시작시각', '평일운영종료시각', '주말운영시작시각',
       '주말운영종료시각', '유료사용여부', '사용기준시간', '사용료', '초과사용단위시간', '초과사용료', '수용가능인원수',
       '면적', '부대시설정보', '신청방법구분', '시설사진정보', '소재지도로명주소', '소재지지번주소', '관리기관명',
       '담당부서명', '사용안내전화번호', '홈페이지주소', '위도', '경도', '데이터기준일자', '제공기관코드',
       '제공기관명'],
      dtype='object')